# Semantic Kernel 
En este ejemplo de código, utilizará el marco de inteligencia artificial  [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) para crear un agente básico.

El objetivo de este ejemplo es mostrarle los pasos que usaremos más adelante en los ejemplos de código adicionales al implementar los diferentes patrones de agencia.

## Importar los paquetes de Python necesarios

In [2]:
import json
import os

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creando el cliente

En este ejemplo lo utilizaremo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acceder al LLM.

El `ai_model_id` se define como `gpt-4o-mini`. Intenta cambiar el modelo a otro disponible en la tienda de modelos de GitHub para ver los diferentes resultados.

Para usar el SDK de inferencia de Azure, que se usa para la URL base de los modelos de GitHub, usaremos el conector OpenAIChatCompletion dentro del kernel semántico. También existen otros [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) utilizar el núcleo semántico para otros proveedores de modelos.

In [3]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """Una lista de destinos aleatorios para unas vacaciones."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Proporciona un destino de vacaciones aleatorio.")
    def get_random_destination(self) -> Annotated[str, "Devuelve un destino de vacaciones aleatorio."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [4]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Crear un servicio de IA que será utilizado por el `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creación del agente

A continuación, creamos el agente llamado `TravelAgent`.

Para este ejemplo, usamos instrucciones muy sencillas. Puedes modificarlas para ver cómo responde el agente de forma diferente.

In [5]:
AGENT_INSTRUCTIONS = """Eres un agente de IA útil que puede ayudar a planificar las vacaciones de los clientes.

Importante: Cuando los usuarios especifiquen un destino, planifique siempre para esa ubicación. Sugiera destinos aleatorios solo si el usuario no ha especificado ninguna preferencia.

Al iniciar la conversación, preséntate con este mensaje:
"¡Hola! Soy tu asistente de agente de viajes. Puedo ayudarte a planificar tus vacaciones y sugerirte destinos interesantes. Puedes preguntarme lo siguiente:
1. Planificar una excursión de un día a un lugar específico.
2. Sugerir un destino de vacaciones al azar.
3. Buscar destinos con características específicas (playas, montañas, sitios históricos, etc.).
4. Planificar un viaje alternativo si no te convence mi primera sugerencia.

¿Qué tipo de viaje te gustaría que te ayudara a planificar hoy?

Prioriza siempre las preferencias de los usuarios. Si mencionan un destino específico como "Bali" o "París", centra tu planificación en ese lugar en lugar de sugerir alternativas."""

agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions=AGENT_INSTRUCTIONS,
)

## Ejecutando los agentes

Ahora podemos ejecutar el Agente definiendo el `ChatHistory` y añadiéndole `system_message`. Usaremos las `AGENT_INSTRUCTIONS` que definimos anteriormente.

Una vez definidos, creamos un `user_inputs` que será lo que el usuario envíe al agente. En este caso, hemos configurado este mensaje como `Planifícame unas vacaciones soleadas`.

Siéntete libre de cambiar este mensaje para ver cómo el agente responde de manera diferente.

In [6]:
user_inputs = [
    "Planifícame una excursión de un día..",
    "No me gusta ese destino. Planifícame otras vacaciones.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()